In [ ]:
import csv
import os
import numpy as np
from numba import njit
import pandas as pd

In [ ]:
#################### FUNCIONES ##################################
def load_mats(path, verbose=False):
    """
    Leer archivos .dat
        1ra matriz -> distancias
        2da matriz -> flujos
    """
    try:
        with open(path, "r") as file:
            lines = file.readlines()
            try:
                mat_size = int(lines[0].strip())
                if (verbose): print(f"mat_size={mat_size}")

                n_mats = int((len(lines)-1) / mat_size)
                if (verbose): print(f"nmats={n_mats}")
                mats = np.zeros((n_mats, mat_size, mat_size), dtype=np.int32)
                for idx_mat in range(n_mats):
                    for i in range(1, mat_size+1):
                        elements = list(filter(None, lines[idx_mat*mat_size + i].strip().split(' ')))
                        if elements:
                            if len(elements) != mat_size:
                                print(f"Matriz irregular [{len(elements)} != {mat_size}] en {path} (linea {idx_mat*mat_size + i})\nDetail:\n{elements}")
                                return dict()
                            for j in range(len(elements)):
                                if elements[j]:
                                    mats[idx_mat, i-1, j] = np.int32(elements[j])
                return {"nombre" : path, "distancia" : mats[0, :, :], "flujo" : mats[1, :, :]}
            except ValueError:
                print("ValueError")
            except:
                print(f"Error al parsear {path}")
    except:
        print(f"Archivo {path} no existe")

    return dict()

def save(path, solucion, valor, iteraciones):
    try:
        file = open(path, "w")

        file.write(f"{valor} {iteraciones}\n")
        for el in solucion:
            file.write(f"{el} ")
        file.write("\n")
    except:
        print(f"Unable to write file {path}")

# Funcion objetivo de kevin
def funcion_objetivo_kevin(solucion, matD, matF):
    permutado = matF[solucion, :][:, solucion]
    return np.sum(np.multiply(matD, permutado), dtype=np.int32)

@njit
def funcion_objetivo(solucion, matD, matF):
    result = 0
    n = matD.shape[0]

    for i in range(n):
        for j in range(n):
                if i != j:
                    result += matD[i, j] * matF[solucion[i] , solucion[j] ]
    return result

@njit
def delta(S, d, f, r, s):
    n = len(S)
    delta_val = 0
    for k in range(n):
        if k == r or k == s:
            continue
        delta_val += (
            d[r][k] * (f[S[r]][S[k]] - f[S[s]][S[k]]) +
            d[k][r] * (f[S[k]][S[r]] - f[S[k]][S[s]]) +
            d[s][k] * (f[S[s]][S[k]] - f[S[r]][S[k]]) +
            d[k][s] * (f[S[k]][S[s]] - f[S[k]][S[r]])
        )
    delta_val += (
        d[r][s] * (f[S[r]][S[s]] - f[S[s]][S[r]]) +
        d[s][r] * (f[S[s]][S[r]] - f[S[r]][S[s]])
    )
    return delta_val

def delta_numpy(S, d, f, r, s):
    S = np.asarray(S)
    n = len(S)

    k = np.array([k for k in range(n) if k != r and k != s])

    # Preextraer ubicaciones
    Sr, Ss = S[r], S[s]
    Sk = S[k]

    # Vectorizados
    term1 = d[r, k] * (f[Sr, Sk] - f[Ss, Sk])
    term2 = d[k, r] * (f[Sk, Sr] - f[Sk, Ss])
    term3 = d[s, k] * (f[Ss, Sk] - f[Sr, Sk])
    term4 = d[k, s] * (f[Sk, Ss] - f[Sk, Sr])

    # Suma total de los cuatro vectores
    delta_val = np.sum(term1 + term2 + term3 + term4, dtype=np.int32)

    # Términos fuera del bucle
    delta_val += d[r][s] * (f[Sr][Ss] - f[Ss][Sr]) + d[s][r] * (f[Ss][Sr] - f[Sr][Ss])

    return delta_val

def generar_solucion_inicial(n : int) -> np.ndarray:
    return np.random.permutation(n, dtype=np.uint8)

def operador_sublista_aleatoria(solucion, s : int):
    """
    Operador de generacion de vecino -> Sublista Aleatoria de Tamaño Fijo:
    este proceso consiste en generar aleatoriamente dos posiciones
     que determinen una sublista de tamaño s, analizar las asignaciones
    efectuadas entre ambas y reasignarlas aleatoriamente. Se considerarán
    sublistas cíclicas y, por tanto, la posición inicial de la sublista
    no siempre será inferior a la posición final.
    """
    n = len(solucion)
    inicio = np.random.randint(n)

    # Crear los índices cíclicos de la sublista
    indices = [(inicio + i) % n for i in range(s)]

    # Extraer y mezclar la sublista
    sublista = [solucion[i] for i in indices]
    shuffled = sublista.copy()
    while shuffled == sublista:
        np.random.shuffle(shuffled)

    # Reinsertar la sublista mezclada
    nueva_solucion = solucion.copy()
    for idx, val in zip(indices, shuffled):
        nueva_solucion[idx] = val

    return nueva_solucion

def vector_to_str(vct : list) -> str:
    s = " "
    for e in vct:
        s = s + str(e) + " "

    return s

In [ ]:
class Logger:

    def __init__(self, ruta_csv, overwrite=True):
        self.ruta_csv = ruta_csv
        self.dataset = None

        # Crear archivo con cabecera
        if overwrite or not os.path.exists(self.ruta_csv):
            with open(self.ruta_csv, mode='w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["Dataset", "Algoritmo", "Semilla", "Costo", "Evaluaciones", "Solucion"])
        else:
            self.data = pd.read_csv(self.ruta_csv)

    def set_dataset(self, dataset):
        self.dataset = dataset

    def log(self, algoritmo, semilla, costo, evaluaciones, solucion):
        if not self.dataset:
            raise Exception("Dataset not set!")
        fila = [
            self.dataset,
            algoritmo,
            semilla,
            costo,
            evaluaciones,
            solucion
        ]

        with open(self.ruta_csv, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(fila)


In [ ]:
########################################### BUSQUEDA LOCAL
def bl_mejor_vecino(seed, matD, matF, funcion_objetivo, delta):
    # Init semilla
    np.random.seed(seed)

    evals = 1

    n_nodes = matD.shape[0]
    mejor_solucion = np.random.permutation(n_nodes)
    mejor_valor = funcion_objetivo(mejor_solucion, matD, matF)
    valor_mejor_vecino = mejor_valor
    seguir = True

    while seguir:
        mejor_i = 0
        mejor_j = 0
        for i in range(n_nodes - 1):
            for j in range(i + 1, n_nodes):
                valor_vecino = mejor_valor - delta(mejor_solucion, matD, matF, i, j)
                #if evals % 10000 == 0: print(evals)
                evals += 1

                if valor_vecino < valor_mejor_vecino:
                    mejor_i = i
                    mejor_j = j
                    valor_mejor_vecino = valor_vecino

        if valor_mejor_vecino < mejor_valor:
            mejor_valor = valor_mejor_vecino
            mejor_solucion[mejor_i], mejor_solucion[mejor_j] = mejor_solucion[mejor_j], mejor_solucion[mejor_i]
        else:
            seguir = False

    return mejor_solucion, valor_mejor_vecino, evals

def bl_primer_mejor(solucion_actual, matD, matF, delta, valor_fo=None, funcion_objetivo=None):
    """
    Se hace la busqueda local del primer mejor
    No hay componentes aleatorias
    """
    n_nodes = matD.shape[0]

    mejor_solucion = solucion_actual
    mejor_valor = valor_fo
    if not valor_fo:
        mejor_valor = funcion_objetivo(mejor_solucion, matD, matF)

    evals = 0

    while True:

        encontrado = False
        for i in range(n_nodes - 1):
            for j in range(i + 1, n_nodes):
                valor_vecino = mejor_valor - delta(mejor_solucion, matD, matF, i, j)
                #valor_vecino = funcion_objetivo(vecino, matD, matF)
                evals += 1
                #print(evals)

                if valor_vecino < mejor_valor:
                    mejor_solucion[i], mejor_solucion[j] = mejor_solucion[j], mejor_solucion[i]
                    mejor_valor = valor_vecino
                    encontrado = True
                    break

            if encontrado:
                break

        if not encontrado:
            break

    return mejor_solucion, mejor_valor, evals

In [ ]:
####################################################### BUSQUEDA GREEDY
def busqueda_greedy(matD, matF):
    """
    Construye una solución inicial para el QAP usando una heurística greedy probabilística.

    :param flujo: Matriz de flujo entre unidades (numpy array n x n).
    :param distancia: Matriz de distancia entre localizaciones (numpy array n x n).
    :return: solucion.
    """
    n = matD.shape[0]
    idx_localizaciones_no_asignadas = np.arange(0, n, dtype=np.uint8)
    idx_unidades_no_asignadas = np.arange(0, n, dtype=np.uint8)

    # Calcular potenciales
    promedio_distancias = np.sum(matD, axis=1)  # Distancia total por localización
    promedio_flujos = np.sum(matF, axis=1) # Flujo total por unidad

    solucion = np.zeros(n, dtype=np.uint8)

    idx_mejores_distancias = sorted(idx_localizaciones_no_asignadas, key=lambda i: promedio_distancias[i])
    idx_mejores_flujos = sorted(idx_unidades_no_asignadas, key=lambda i: promedio_flujos[i], reverse=True)

    while idx_localizaciones_no_asignadas.size != 0:
        # Selección de los mejores candidatos
        idx_localizacion_mejor = idx_mejores_distancias.pop(0)
        idx_unidad_mejor = idx_mejores_flujos.pop(0)

        #solucion[idx_localizacion_mejor] = idx_unidad_mejor
        # Segun ChatGPT es asi
        solucion[idx_unidad_mejor] = idx_localizacion_mejor

        idx_localizaciones_no_asignadas = np.delete(idx_localizaciones_no_asignadas, np.where(idx_localizaciones_no_asignadas == idx_localizacion_mejor))
        idx_unidades_no_asignadas = np.delete(idx_unidades_no_asignadas, np.where(idx_unidades_no_asignadas == idx_unidad_mejor))

    return solucion


In [ ]:
################################################## GRASP

def generar_greedy_prob(matD, matF, l : int, seed : float):
    """
    Construye una solución inicial para el QAP usando una heurística greedy probabilística.

    :param flujo: Matriz de flujo entre unidades (numpy array n x n).
    :param distancia: Matriz de distancia entre localizaciones (numpy array n x n).
    :param l: Tamaño de las listas RCL (listas de candidatos restringidas).
    :return: Diccionario representando la asignación {unidad: localización}.
    """
    # Init semilla
    np.random.seed(seed)

    n = matD.shape[0]
    idx_localizaciones_no_asignadas = np.arange(0, n, dtype=np.uint8)
    idx_unidades_no_asignadas = np.arange(0, n, dtype=np.uint8)

    promedio_distancias = np.sum(matD, axis=1)  # Distancia total por localización
    promedio_flujos = np.sum(matF, axis=1) # Flujo total por unidad

    solucion = np.zeros(n, dtype=np.uint8)

    while idx_localizaciones_no_asignadas.size != 0:
        idx_mejores_distancias = sorted(idx_localizaciones_no_asignadas, key=lambda i: promedio_distancias[i])
        l_mejores_distancias = idx_mejores_distancias[:min(l, len(idx_mejores_distancias))]

        idx_mejores_flujos = sorted(idx_unidades_no_asignadas, key=lambda i: -promedio_flujos[i])
        l_mejores_flujos = idx_mejores_flujos[:min(l, len(idx_mejores_flujos))]

        # Selección aleatoria de entre los mejores candidatos
        idx_localizacion_aleatoria = np.random.choice(l_mejores_distancias)
        idx_unidad_aleatoria = np.random.choice(l_mejores_flujos)

        # Segun ChatGPT es asi
        solucion[idx_unidad_aleatoria] = idx_localizacion_aleatoria

        # Se eliminan de las listas
        #idx_localizaciones_no_asignadas.remove(idx_localizacion_aleatoria)
        #idx_unidades_no_asignadas.remove(idx_unidad_aleatoria)

        idx_localizaciones_no_asignadas = np.delete(idx_localizaciones_no_asignadas, np.where(idx_localizaciones_no_asignadas == idx_localizacion_aleatoria))
        idx_unidades_no_asignadas = np.delete(idx_unidades_no_asignadas, np.where(idx_unidades_no_asignadas == idx_unidad_aleatoria))

    return solucion, funcion_objetivo(solucion, matD, matF)


############################### Algoritmo GRASP ###############################################
def GRASP(matD, matF, l, semillas_bl, logger : Logger = None):
    mejor_solucion = None
    mejor_valor = np.inf
    evaluaciones = 0

    for seed_bl in semillas_bl:
        solucion_greedy, valor_greedy = generar_greedy_prob(matD, matF, l, seed_bl)
        if (logger != None): logger.log("Greedy Prob", seed_bl, valor_greedy, 1, vector_to_str(solucion_greedy))

        solucion_optimizada, valor_optimizada, eval = bl_primer_mejor(solucion_greedy, matD, matF, delta, valor_fo=valor_greedy)
        if (logger != None): logger.log("BL Primer Mejor", "N/A", valor_optimizada, eval, vector_to_str(solucion_optimizada))
        evaluaciones += eval

        if valor_optimizada < mejor_valor:
            mejor_solucion = solucion_optimizada
            mejor_valor = valor_optimizada

    return mejor_solucion, mejor_valor, evaluaciones

In [ ]:
################################################### BUSQUEDA ILS

def ILS(matD, matF, seed, logger : Logger = None):
    # Init semilla
    np.random.seed(seed)

    n = matD.shape[0]
    s = int(np.round(n/4))
    evaluaciones = 0

    solucion_actual = np.random.permutation(n)
    valor_actual = funcion_objetivo(solucion_actual, matD, matF)

    mejor_solucion = solucion_actual
    mejor_valor = valor_actual

    """
    PARA HACER ILS POBLACIONAL:
    HACER UN VECTOR DE SOLUCIONES : [s1, s2, s3 .. sK]
    Optimizar cada solucion con busqueda local y luego seleccionar y obtener el vector final:

    vector final = [mejor(s1, s1_opt), .. mejor(sK, sK_opt)]
    """

    num_no_mejora = 0
    while num_no_mejora < 10:
        solucion_optimizada, valor_optimizado, evals = bl_primer_mejor(solucion_actual, matD, matF, delta, funcion_objetivo=funcion_objetivo)
        evaluaciones += evals

        if valor_optimizado < mejor_valor:
            mejor_solucion = solucion_optimizada
            mejor_valor = valor_optimizado
            num_no_mejora = 0
        else:
            num_no_mejora += 1

        # Hacer mutacion sobre mejor_solucion
        solucion_actual = operador_sublista_aleatoria(mejor_solucion, s)

    return mejor_solucion, mejor_valor, evaluaciones

In [ ]:
################################################### BUSQUEDA VNS

def VNS(matD, matF, seed):
    # Init semilla
    np.random.seed(seed)

    n = matD.shape[0]
    k_max = 5
    bl_max = 10
    k = 1
    bl = 0

    mejor_solucion = np.random.permutation(n)
    mejor_valor = funcion_objetivo(mejor_solucion, matD, matF)
    evaluaciones = 0

    # Paso 6
    while bl < bl_max:
        # Paso 2
        if k > k_max:
            k = 1
        # Paso 3
        """
        k = 1: Se aplica un tamaño de s = n/8.
        k = 2: Se aplica un tamaño de s = n/7.
        k = 3: Se aplica un tamaño de s = n/6.
        k = 4: Se aplica un tamaño de s = n/5.
        k = 5: Se aplica un tamaño de s = n/4.
        """
        s = int(np.round(n / (9 - k)))
        vecino = operador_sublista_aleatoria(mejor_solucion, s)
        # Paso 4
        vecino_opt, valor_vecino, evals = bl_primer_mejor(vecino, matD, matF, delta, funcion_objetivo=funcion_objetivo)
        evaluaciones += evals
        bl += 1
        # Paso 5
        if valor_vecino < mejor_valor:
            mejor_solucion = vecino_opt
            mejor_valor = valor_vecino
            k = 1
        else:
            k += 1

    return mejor_solucion, mejor_valor, evaluaciones

In [ ]:
grasp_logger = Logger("resultados/GRASP.csv")
tai25 = load_mats("datasets/tai25b.dat")
sko90 = load_mats("datasets/sko90b.dat")
tai150 = load_mats("datasets/tai150b.dat")
main_logger = Logger("resultados/practica2.csv")


for d in [tai25]: # Datasets QAP
    main_logger.set_dataset(d['nombre'])
    grasp_logger.set_dataset(d['nombre'])

    n = d['distancia'].shape[0]
    matD = d['distancia']
    matF = d['flujo']

    grasp_logger.archivo = d['nombre']
    seeds = [42, 420, 4200, 42000, 80987]

    # Greedy
    solucion = busqueda_greedy(matD, matF)
    valor = funcion_objetivo(solucion, matD, matF)
    main_logger.log("Greedy", "N/A", valor, 1, vector_to_str(solucion))

    # Busqueda Local
    #seeds = [42] # PARA TESTEAR
    for seed_bl in seeds:
        solucion, valor, evaluaciones = bl_mejor_vecino(seed_bl, matD, matF, funcion_objetivo, delta)
        main_logger.log("BL Mejor Vecino", seed_bl, valor, evaluaciones, vector_to_str(solucion))


    # GRASP
    l = int(np.round(0.1 * n))
    solucion, valor, evaluaciones = GRASP(matD, matF, l, seeds, grasp_logger)
    main_logger.log("GRASP", vector_to_str(seeds), valor, evaluaciones, vector_to_str(solucion))

    # ILS
    seed = 42
    solucion, valor, evaluaciones = ILS(matD, matF, seed)
    main_logger.log("ILS", seed, valor, evaluaciones, vector_to_str(solucion))

    # VNS
    for seed in seeds:
        solucion, valor, evaluaciones = VNS(matD, matF, seed)
        main_logger.log("VNS", seed, valor, evaluaciones, vector_to_str(solucion))
